In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

import os
import cv2

In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = 0.4
session = InteractiveSession(config=config)

In [4]:
image_size = 227

groups_folder_path = '../Data/AAFT_20/'

X_r = []
y = []

for label in ['Bi', 'Tri']:

    for feature in ['striping_bw']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            for filename in f:
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])

                X_r.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                else:
                    y.append(1)

print(X_r[0].shape)

(227, 227, 3)


In [5]:
X = np.array(X_r)
y = np.array(y)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# validation split

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)

In [7]:
AlexNet = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding="same", activation="relu", input_shape=(227, 227, 3)),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides= 2),

    keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

    keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

    keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

    keras.layers.Flatten(),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

In [8]:
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 227, 227, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 227, 227, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 113, 113, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 113, 113, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 113, 113, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [9]:
AlexNet.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
AlexNet_history = AlexNet.fit(X_train, y_train, epochs=10, batch_size=16,
                    validation_data=(X_validation, y_validation))

Train on 1728 samples, validate on 192 samples
Epoch 1/10
1728/1728 [==============================] - 22s 13ms/sample - loss: 0.7306 - accuracy: 0.4988 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 2/10
1728/1728 [==============================] - 18s 10ms/sample - loss: 0.6937 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
1728/1728 [==============================] - 18s 10ms/sample - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
1728/1728 [==============================] - 18s 10ms/sample - loss: 0.6933 - accuracy: 0.4815 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
1728/1728 [==============================] - 18s 10ms/sample - loss: 0.6939 - accuracy: 0.4884 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
1728/1728 [==============================] - 18s 10ms/sample - loss: 0.6935 - accuracy: 0.4942 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/10
1728/1728 [==============================] - 17s 10ms/

In [ ]:
evaluation = AlexNet.evaluate(X_train, y_train, batch_size=16)

In [ ]:
evaluation = AlexNet.evaluate(X_train, y_train, batch_size=16)